In [60]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
# Path ke direktori dataset
dataset_dir = '/content/drive/My Drive/capstone/cobalagi/'

In [66]:
# Membaca dataset dan melakukan ekstraksi fitur menggunakan FLBP
def extract_features(dataset_dir):
    images = []
    labels = []

    for filename in os.listdir(dataset_dir):
        if filename.endswith('.bmp'):
            image_path = os.path.join(dataset_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (100, 100))  # Mengubah ukuran gambar menjadi 100x100 pixel
            
            # Normalisasi gambar
            image = image.astype(float) / 255.0
            
            # Ekstraksi fitur menggunakan FLBP
            radius = 3
            n_points = 16 * radius
            lbp = local_binary_pattern(image, n_points, radius, method='uniform')
            feature = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))[0]
            
            images.append(feature)
            
            # Mendapatkan angka kolestrol dari nama file
            cholesterol = filename.split('-')[1]
            if cholesterol.isdigit():
                labels.append(int(cholesterol))
            else:
                print("Invalid filename format:", filename)

    return np.array(images), np.array(labels)

In [67]:
# Memanggil fungsi untuk melakukan ekstraksi fitur dari dataset menggunakan FLBP
images, labels = extract_features(dataset_dir)

In [68]:
images

array([[ 382,  144,   96, ...,   97,  399, 4412],
       [ 364,   84,   75, ...,   70,  335, 3564],
       [ 382,  144,   96, ...,   97,  399, 4412],
       ...,
       [ 312,  118,   91, ...,   85,  307, 4045],
       [ 312,  118,   91, ...,   85,  307, 4045],
       [ 298,  126,   96, ...,   51,  277, 3784]])

In [69]:
# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [70]:
# Membangun model regresi linear
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [71]:
LinearRegression()


LinearRegression()

In [87]:
# Evaluasi model
score = model.score(X_test, y_test)
print("Akurasi model:", score)

Akurasi model: 0.8993623374899622


# New Section

In [73]:
# Lakukan prediksi pada data uji
y_pred = model.predict(X_test)

# Ini untuk 3 Class
threshold_low = 200
threshold_high = 240

# Mengkonversi prediksi menjadi kelas berdasarkan threshold
y_pred_classes = np.zeros_like(y_pred)  # Inisialisasi dengan kelas rendah
y_pred_classes[y_pred >= threshold_high] = 2  # Kelas tinggi
y_pred_classes[(y_pred >= threshold_low) & (y_pred < threshold_high)] = 1  # Kelas beresiko

y_test_classes = np.zeros_like(y_test)
y_test_classes[y_test >= threshold_high] = 2
y_test_classes[(y_test >= threshold_low) & (y_test < threshold_high)] = 1

# Evaluasi model
confusion_mat = confusion_matrix(y_test_classes, y_pred_classes)
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

In [74]:
print("Confusion Matrix:")
print(confusion_mat)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Confusion Matrix:
[[2 0 0]
 [0 8 0]
 [0 0 5]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [79]:
from google.colab.patches import cv2_imshow
# Path gambar yang ingin diperiksa
image_path = '/content/drive/My Drive/capstone/cobalagi/kolesterol-247-14-kiri.bmp'

# Membaca gambar menggunakan cv2.imread()
image = cv2.imread(image_path)

# Memeriksa apakah gambar berhasil dibaca
if image is not None:
    print("Gambar berhasil dibaca.")
    #cv2.imshow("Gambar", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Gagal membaca gambar.")

Gambar berhasil dibaca.


In [80]:
image = cv2.imread(image_path)
image = cv2.resize(image, (200, 200))
#cv2.imshow("Gambar", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [81]:
from skimage.feature import local_binary_pattern

In [83]:
# Coba Predict data
# Memuat gambar
image_path = '/content/drive/My Drive/capstone/cobalagi/kolesterol-247-14-kiri.bmp'
image = cv2.imread(image_path)

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))  # Mengubah ukuran gambar menjadi 100x100 pixel
            
# Ekstraksi fitur menggunakan FLBP
radius = 3
n_points = 16 * radius
lbp = local_binary_pattern(image, n_points, radius, method='uniform')
feature = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))[0]

# Mengubah dimensi fitur menjadi bentuk yang dapat diterima oleh model (sesuaikan dengan bentuk fitur yang digunakan saat melatih model)
feature = np.reshape(feature, (1, -1))

# Melakukan prediksi menggunakan model
prediction = model.predict(feature)

# Cetak hasil prediksi
print("Prediksi Kolesterol:", prediction)

Prediksi Kolesterol: [246.09244191]


In [84]:
pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Save Model menggunakan joblib
import joblib

# Simpan model ke dalam file .h5 menggunakan joblib
joblib.dump(model, 'regression_model.h5')

['regression_model.h5']

In [85]:
['regression_model.h5']

['regression_model.h5']

In [86]:
# Muat kembali model dari file .h5
model = joblib.load('regression_model.h5')